In [ ]:
!pip install torch_snippets
from torch_snippets import inspect, show, np, torch, nn
from torchvision.models import resnet50
model=resnet50(pretrained=True)
for param in model.parameters():
  param.requires_grad=False
model.eval()
import requests
from PIL import Image

url= 'https://lionsvalley.co.za/wp-content/uploads/2015/11/africanelephant-square.jpg'
original_image=Image.open(requests.get(url, stream=True).raw).convert('RGB')
original_image=np.array(original_image)
original_image=torch.Tensor(original_image)

In [ ]:
image_net_classes== 'https://gist.githubusercontent.com/ \
yrevar/942d3a0ac09ec9e5eb3a/raw/238f720ff059c1f82f368259d1ca4ffa5dd8f9f5/ \
imagenet1000_clsidx_to_labels.txt'
image_net_classes=requests.get(image_net_classes).text
image_net_ids=eval(image_net_classes)
image_net_classes={i:j for j, i in image_net_ids.item()}

In [ ]:
#@ function to normalize(img2tensor) and denormalize(tensor2img):
from torchvision import transforms as T
from torch.nn import functional as f

normalize=T.Normalize([0.485, 0.456, 0.406])

denormalize=T.Normalize(
    [-0.485/0.229, -0.456/0.224, -0.406/0.225], #reversing mean normalization
    [1/0.229, 1/0.224, 1/0.225] #reversing std
)

def Image2tensor(input):
  x=normalize(input.clone().permute(2, 0, 1)/255.)[None]
  return x

def tensor2Image(input):
  x=(denormalize(input[0].clone()).permute(1, 2, 0)*255.).type(torch.uint8)
  return x

In [ ]:
def predict_on_target(input):
  model.eval()
  show(input)
  input=Image2tensor(input)
  pred=model(input)
  pred=F.softmax(pred, dim=-1)[0]
  prob, clss=torch.max(pred, 0)
  clss=image_net_ids[clss.item()]
  print(f'PREDICTION: {clss} @ {prob.item()}')